In [4]:
#Fixing absolute and relative paths for folders and notebooks

import os
from pathlib import Path

if Path.cwd().name == '_notebooks':
    os.chdir('..')  # Go to project root
elif not (Path.cwd() / '__notebooks').exists():
    # Find and change to project root
    for parent in Path.cwd().parents:
        if (parent / 'notebooks').exists():
            os.chdir(parent)
            break

print(f"Working directory set to: {Path.cwd()}")

Working directory set to: /Users/allegratuvida/Desktop/escritorio/data_critica_2025/Proyectos en desarrollo/Mapeos comunitarios/mapeos_comunitarios


In [ ]:
import folium
from folium.features import CustomIcon
from IPython.display import display
import requests

# 1. Select a background map (OpenStreetMap by default)
map_center = resistencias_df['_geolocation'].dropna().apply(lambda x: x if isinstance(x, list) and len(x) == 2 else None).dropna().tolist()
if map_center:
    avg_lat = sum([loc[0] for loc in map_center]) / len(map_center)
    avg_lon = sum([loc[1] for loc in map_center]) / len(map_center)
else:
    avg_lat, avg_lon = 19.4326, -99.1332  # Default to Mexico City

m = folium.Map(location=[avg_lat, avg_lon], zoom_start=6, tiles='OpenStreetMap')

# 2. Define a custom icon (can be replaced with a local or web image)
icon_url = 'https://cdn-icons-png.flaticon.com/512/684/684908.png'  # Example icon

# 3. Add markers with popups
for idx, row in resistencias_df.iterrows():
    geo = row.get('_geolocation')
    if not (isinstance(geo, list) and len(geo) == 2 and all(isinstance(x, (float, int)) for x in geo)):
        continue
    lat, lon = geo
    # Retrieve image from Kobo asset if available
    image_url = row.get('A_ade_la_primera_foto_que_ilus', None)
    image_html = ''
    if image_url:
        # Kobo media URLs are usually relative, you may need to prepend the server URL
        if not image_url.startswith('http'):
            image_url = f"https://eu.kobotoolbox.org/media/original?media_file={image_url}"
        image_html = f'<br><img src="{image_url}" width="200">'
    # Prepare popup HTML
    popup_html = f"""
    <b>{row.get('_Cu_l_es_el_nombre_d_ue_quieres_registrar', '')}</b><br>
    Fecha: {row.get('Selecciona_la_fecha_resistencia_en_2021', '')}<br>
    Pueblo/Colección: {row.get('_A_qu_pueblo_o_cole_cci_n_en_resistencia', '')}<br>
    Descripción: {row.get('Describe_en_un_p_rra_lo_m_s_breve_posible', '')}<br>
    {image_html}
    """
    icon = CustomIcon(icon_url, icon_size=(30, 30))
    folium.Marker(
        location=[lat, lon],
        icon=icon,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m)

# 4. Display the map
m